# Practical Exam: House sales

RealAgents is a real estate company that focuses on selling houses.

RealAgents sells a variety of types of house in one metropolitan area.

Some houses sell slowly and sometimes require lowering the price in order to find a buyer.

In order to stay competitive, RealAgents would like to optimize the listing prices of the houses it is trying to sell.

They want to do this by predicting the sale price of a house given its characteristics.

If they can predict the sale price in advance, they can decrease the time to sale.


## Data

The dataset contains records of previous houses sold in the area.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| house_id    | Nominal. </br> Unique identifier for houses. </br>Missing values not possible. |
| city        | Nominal. </br>The city in which the house is located. One of 'Silvertown', 'Riverford', 'Teasdale' and 'Poppleton'. </br>Replace missing values with "Unknown". |
| sale_price  | Discrete. </br>The sale price of the house in whole dollars. Values can be any positive number greater than or equal to zero.</br>Remove missing entries. |
| sale_date   | Discrete. </br>The date of the last sale of the house. </br>Replace missing values with 2023-01-01. |
| months_listed  | Continuous. </br>The number of months the house was listed on the market prior to its last sale, rounded to one decimal place. </br>Replace missing values with mean number of months listed, to one decimal place. |
| bedrooms    | Discrete. </br>The number of bedrooms in the house. Any positive values greater than or equal to zero. </br>Replace missing values with the mean number of bedrooms, rounded to the nearest integer. |
| house_type   | Ordinal. </br>One of "Terraced" (two shared walls), "Semi-detached" (one shared wall), or "Detached" (no shared walls). </br>Replace missing values with the most common house type. |
| area      | Continuous. </br>The area of the house in square meters, rounded to one decimal place. </br>Replace missing values with the mean, to one decimal place. |


# Task 1

The team at RealAgents knows that the city that a property is located in makes a difference to the sale price. 

Unfortuntately they believe that this isn't always recorded in the data. 

Calculate the number of missing values of the `city`. 

 - You should use the data in the file "house_sales.csv". 

 - Your output should be an object `missing_city`, that contains the number of missing values in this column. 

In [10]:
import pandas as pd

# Load the CSV file into a Pandas DataFrame
houses_df = pd.read_csv("house_sales.csv")

# Count the number of missing values in the "city" column
missing_city = houses_df["city"].isna().sum()
missing_city

np.int64(0)

# Task 2 

Before you fit any models, you will need to make sure the data is clean. 

The table below shows what the data should look like. 

Create a cleaned version of the dataframe. 

 - You should start with the data in the file "house_sales.csv". 

 - Your output should be a dataframe named `clean_data`. 

 - All column names and values should match the table below.


| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| house_id    | Nominal. </br> Unique identifier for houses. </br>Missing values not possible. |
| city        | Nominal. </br>The city in which the house is located. One of 'Silvertown', 'Riverford', 'Teasdale' and 'Poppleton' </br>Replace missing values with "Unknown". |
| sale_price  | Discrete. </br>The sale price of the house in whole dollars. Values can be any positive number greater than or equal to zero.</br>Remove missing entries. |
| sale_date   | Discrete. </br>The date of the last sale of the house. </br>Replace missing values with 2023-01-01. |
| months_listed  | Continuous. </br>The number of months the house was listed on the market prior to its last sale, rounded to one decimal place. </br>Replace missing values with mean number of months listed, to one decimal place. |
| bedrooms    | Discrete. </br>The number of bedrooms in the house. Any positive values greater than or equal to zero. </br>Replace missing values with the mean number of bedrooms, rounded to the nearest integer. |
| house_type   | Ordinal. </br>One of "Terraced", "Semi-detached", or "Detached". </br>Replace missing values with the most common house type. |
| area      | Continuous. </br>The area of the house in square meters, rounded to one decimal place. </br>Replace missing values with the mean, to one decimal place. |

In [11]:
def transform_data(data: pd.DataFrame) -> pd.DataFrame:
    # Create a copy of the data to ensure we're not working on a view
    df = data.copy()

    # Transform 'city' column
    df["city"] = df["city"].fillna("Unknown").astype("category")

    # Transform 'sale_date' column
    df["sale_date"] = df["sale_date"].fillna("2023-01-01")

    # Transform 'months_listed' column
    mean_months_listed = df["months_listed"].mean()
    df["months_listed"] = df["months_listed"].fillna(round(mean_months_listed, 1))

    # Transform 'bedrooms' column
    mean_bedrooms = df["bedrooms"].mean()
    df["bedrooms"] = df["bedrooms"].fillna(round(mean_bedrooms))

    # Transform 'house_type' column
    category_order = ["Terraced", "Semi-detached", "Detached"]

    # Fill NA values before converting to categorical
    most_common_house_type = df["house_type"].mode().iloc[0]
    df["house_type"] = df["house_type"].fillna(most_common_house_type)

    # Now convert to categorical
    df["house_type"] = pd.Categorical(
        df["house_type"], categories=category_order, ordered=True
    )

    # Transform 'area' column
    df["area"] = df["area"].str.replace(" sq.m.", "", regex=False).astype(float)
    mean_area = df["area"].mean()
    df["area"] = df["area"].fillna(round(mean_area, 1))

    return df

clean_data = transform_data(houses_df)
clean_data

,house_id,city,sale_price,sale_date,months_listed,bedrooms,house_type,area
0,1217792,Silvertown,55943,2021-09-12,5.4,2,Semi-detached,107.8
1,1900913,Silvertown,384677,2021-01-17,6.3,5,Detached,498.8
2,1174927,Riverford,281707,2021-11-10,6.9,6,Detached,542.5
3,1773666,Silvertown,373251,2020-04-13,6.1,6,NaN,528.4
4,1258487,Silvertown,328885,2020-09-24,8.7,5,Detached,477.1
...,...,...,...,...,...,...,...,...
1495,1123892,Riverford,198661,2022-02-17,7.3,5,Detached,432.2
1496,1327295,Poppleton,358304,2020-10-10,6.2,6,Detached,599.8
1497,1058161,Riverford,176612,2022-11-01,6.8,4,Detached,359.1
1498,1822037,Teasdale,197827,2021-04-03,5.4,3,Detached,253.7


# Task 3 

The team at RealAgents have told you that they have always believed that the number of bedrooms is the biggest driver of house price. 

Producing a table showing the difference in the average sale price by number of bedrooms along with the variance to investigate this question for the team.

 - You should start with the data in the file 'house_sales.csv'.

 - Your output should be a data frame named `price_by_rooms`. 

 - It should include the three columns `bedrooms`, `avg_price`, `var_price`. 

 - Your answers should be rounded to 1 decimal place.   

In [12]:
def analyze_price_by_bedrooms(data: pd.DataFrame) -> pd.DataFrame:
    # Convert 'sale_price' to numeric (in case it's not already)
    data['sale_price'] = pd.to_numeric(data['sale_price'], errors='coerce')

    # Calculate average sale price and variance by number of bedrooms
    price_by_rooms = data.groupby('bedrooms')['sale_price'].agg(['mean', 'var']).reset_index()
    price_by_rooms = price_by_rooms.rename(columns={'mean': 'avg_price', 'var': 'var_price'})

    # Round the values to one decimal place
    price_by_rooms['avg_price'] = price_by_rooms['avg_price'].round(1)
    price_by_rooms['var_price'] = price_by_rooms['var_price'].round(1)

    return price_by_rooms

price_by_rooms = analyze_price_by_bedrooms(houses_df)
price_by_rooms

,bedrooms,avg_price,var_price
0,2,67076.4,5.652896e+08
1,3,154665.1,2.378289e+09
2,4,234704.6,1.725211e+09
3,5,301515.9,2.484328e+09
4,6,375741.3,3.924432e+09


# Task 4

Fit a baseline model to predict the sale price of a house.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `base_result`, that includes `house_id` and `price`. The price column must be your predicted values.

In [13]:
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


def create_baseline_model(train_file: str, validation_file: str) -> pd.DataFrame:
    # Load the data
    train_data = pd.read_csv(train_file)
    validation_data = pd.read_csv(validation_file)

    # Define features to use in the model
    numeric_features = ["bedrooms", "area", "months_listed"]
    categorical_features = ["city", "house_type"]

    # Create preprocessing steps
    numeric_transformer = SimpleImputer(strategy="mean")
    categorical_transformer = Pipeline(
        steps=[
            ("imputer", SimpleImputer(strategy="constant", fill_value="missing")),
            ("onehot", OneHotEncoder(handle_unknown="ignore")),
        ]
    )

    # Combine preprocessing steps
    preprocessor = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, numeric_features),
            ("cat", categorical_transformer, categorical_features),
        ]
    )

    # Create a pipeline with the preprocessor and the model
    model = Pipeline(
        steps=[("preprocessor", preprocessor), ("regressor", LinearRegression())]
    )

    # Fit the model
    model.fit(
        train_data[numeric_features + categorical_features], train_data["sale_price"]
    )

    # Make predictions on the validation set
    predictions = model.predict(
        validation_data[numeric_features + categorical_features]
    )

    # Create the result dataframe
    base_result = pd.DataFrame(
        {
            "house_id": validation_data["house_id"],
            "price": np.round(predictions, 0).astype(int),
        }
    )

    return base_result

train_file = "train.csv"
validation_file = "validation.csv"
base_result = create_baseline_model(train_file, validation_file)
base_result

,house_id,price
0,1331375,121528
1,1630115,304387
2,1645745,384760
3,1336775,123976
4,1888274,271186
...,...,...
295,1986255,351916
296,1896276,370139
297,1758223,259024
298,1752010,169120


# Task 5

Fit a comparison model to predict the sale price of a house.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `compare_result`, that includes `house_id` and `price`. The price column must be your predicted values.

In [14]:
from sklearn.ensemble import RandomForestRegressor

def create_comparison_model(train_file: str, validation_file: str) -> pd.DataFrame:
    # Load the data
    train_data = pd.read_csv(train_file)
    validation_data = pd.read_csv(validation_file)

    # Define features to use in the model
    numeric_features = ["bedrooms", "area", "months_listed"]
    categorical_features = ["city", "house_type"]

    # Create preprocessing steps
    numeric_transformer = SimpleImputer(strategy="median")
    categorical_transformer = Pipeline(
        steps=[
            ("imputer", SimpleImputer(strategy="constant", fill_value="missing")),
            ("onehot", OneHotEncoder(handle_unknown="ignore")),
        ]
    )

    # Combine preprocessing steps
    preprocessor = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, numeric_features),
            ("cat", categorical_transformer, categorical_features),
        ]
    )

    # Create a pipeline with the preprocessor and the model
    model = Pipeline(
        steps=[
            ("preprocessor", preprocessor),
            ("regressor", RandomForestRegressor(n_estimators=100, random_state=42)),
        ]
    )

    # Fit the model
    model.fit(
        train_data[numeric_features + categorical_features], train_data["sale_price"]
    )

    # Make predictions on the validation set
    predictions = model.predict(
        validation_data[numeric_features + categorical_features]
    )

    # Create the result dataframe
    compare_result = pd.DataFrame(
        {
            "house_id": validation_data["house_id"],
            "price": np.round(predictions, 0).astype(int),
        }
    )

    return compare_result

train_file = "train.csv"
validation_file = "validation.csv"
compare_result = create_comparison_model(train_file, validation_file)

compare_result

,house_id,price
0,1331375,82491
1,1630115,302167
2,1645745,404716
3,1336775,106530
4,1888274,267662
...,...,...
295,1986255,363699
296,1896276,393525
297,1758223,264338
298,1752010,178309


View the root mean square error of the comparison model. It should be less than 30000.

In [15]:
import math
import numpy as np

y1 = base_result["price"]
y2 = compare_result["price"]

MSE = np.square(np.subtract(y1, y2)).mean()

RMSE = math.sqrt(MSE)
print(f"Root Mean Square Error: {RMSE}")

Root Mean Square Error: 18159.217198253158
